In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz

import difflib 
from fuzzywuzzy import process


### Feb 17 todo list:

- import basics from extract flora code
- organize fuzzy matching authors 
    - function for splitting authors into sets
    - functions for exact matching species name and fuzzy matching authors
    - functions for getting the match types 

### importing books and WCVP

In [2]:
# to fix unnecessary space at the end of authors (will fix index files later)
def fix_author(row):
    if not isinstance(row, str) or row[-1] != ' ':
        return row
    return row[:-1]

In [7]:
vol1_path = '../input/input copy/index_without_itallics_csvs/vol1/vol1_index_pruned.csv'
vol2_path = '../input/input copy/index_without_itallics_csvs/vol2/vol2_index_pruned.csv'
vol3_path = '../input/input copy/index_without_itallics_csvs/vol3/vol3_index_pruned.csv'

vol1_df = pd.read_csv(vol1_path)
vol2_df = pd.read_csv(vol2_path)
vol3_df = pd.read_csv(vol3_path)

#fix spacing issue
vol1_df['author'] = vol1_df['author'].apply(fix_author)
vol2_df['author'] = vol2_df['author'].apply(fix_author)
vol3_df['author'] = vol3_df['author'].apply(fix_author)

#changing name of columns 
vol1_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol2_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)
vol3_df.rename(columns={'genus': 'mouterde_genus', 'epithet': 'mouterde_epithet', 'author':'mouterde_author', 'infra':'mouterde_infra'}, inplace=True)

#wcvp 
text_wcvp_path = '../input/input copy/wcvp/wcvp_v9_jun_2022.txt'
wcvp_df = pd.read_csv(text_wcvp_path, sep = '|')

### process wcvp (explode authors)

In [22]:
def get_set(x):
    if not isinstance(x, list): #is this the best way
        x = []
    return set(x)

#to use after exploding the authors
connectors = {'ex', '&', 'et'}
def remove_prantheses(x):
    if isinstance(x, str):
        return re.sub(r'([()])', '', x)
    return x

def process_wcvp(wcvp_df):
    wcvp_df['infraspecies'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['species'].replace(np.nan, 'np.NaN', inplace=True)
    wcvp_df['authors_set'] = wcvp_df['authors'].str.split(' ').apply(lambda x : get_set(x) - connectors)
    wcvp_df_explode = wcvp_df.explode('authors_set')
    wcvp_df_explode['authors_set'] = wcvp_df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return wcvp_df_explode

wcvp_df_explode = process_wcvp(wcvp_df)

### process vols (explore authors)

just doing the set process during matching

In [23]:
"""def process_vols(df):
    delimeter = ' ' #TODO: FIX this absolutely HORRIBLE delimeter 
    df['authors_set'] = df['mouterde_author'].str.split(delimeter).apply(lambda x : get_set(x) - connectors)
    df_explode = df.explode('authors_set')
    df_explode['authors_set'] = df_explode['authors_set'].apply(remove_prantheses) #to remove prantheses
    return df_explode"""

In [26]:
"""vol1_explode = process_vols(vol1_df)
vol2_explode = process_vols(vol2_df)
vol3_explode = process_vols(vol3_df)"""

### Get Closest Match

## UNTITLED 

In [101]:
def get_closest_match(r):
    m_author = r['mouterde_author'] 
    genus = r['mouterde_genus']
    epithet = r['mouterde_epithet']
    infra = r['mouterde_infra']
    wcvp_df_gs_match = wcvp_df[(wcvp_df['genus'] == genus) & (wcvp_df['species'] == epithet) & (wcvp_df['infraspecies'] == infra)]
    
    possible_author = []

    #this doesn't work as is but here's the idea 
    
    if isinstance(m_author, str):
        m_author_set = list(get_set(m_author.split(' ')) - connectors)
        for author in m_author_set:
            #print(author, process.extract(author, wcvp_df_gs_match['authors_set']))
            try:
                author_match, author_score, _ = process.extract(author, wcvp_df_gs_match['authors_set'])[0]
                print(author, author_match, author_score)
                if author_score >= 85:
                    possible_author.append(author_match)
            except:
                possible_author.append(np.nan)
        #author_match = difflib.get_close_matches(m_author, wcvp_df_gs_match['authors'])
    else:
        possible_author.append(np.nan)
    return possible_author

In [102]:
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)


Less.' {'Less.'} 100
Less. {'Less.'} 100
Boiss. {'Boiss.'} 100
(Willd.) {'(Willd.)', 'Boiss.'} 90
Boiss. {'(Willd.)', 'Boiss.'} 90
Mout. {'Mouterde'} 90
Mobayen {'Mobayen'} 100
Boiss. {'Boiss.'} 100
(Schultes) {'(Willd.', 'Boiss.', 'Schult.)'} 77
Boiss. {'(Willd.', 'Boiss.', 'Schult.)'} 90
L. {'L.'} 100
L. {'L.'} 100
L. {'L.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
DC. {'DC.'} 100
np.NaN set() 0
Afan. {'Afanasiev'} 90
DC. {'DC.'} 100
L. {'L.'} 100
Schultz {'Sch.Bip.', '(Forssk.)'} 51
Bip. {'Sch.Bip.', '(Forssk.)'} 90
(Forssk.) {'Sch.Bip.', '(Forssk.)'} 90
Boiss. {'Boiss.'} 100
(Labili.). {'DC.'} 0
DC. {'DC.'} 100
DC. {'DC.'} 100
Koch {'K.Koch'} 90
C. {'K.Koch'} 90
Cass. {'Cass.'} 100
Cass. {'Cass.', '(L.)'} 90
(L.) {'Cass.', '(L.)'} 60
L. {'L.'} 100
Davis {'P.H.Davis'} 90
Schreb. {'Schreb.'} 100
Boiss. {'Boiss.'} 100
L. {'L.'} 100
Bilik {'Bilik'} 100
Benth. {'Benth.', 'Ging.'} 90
Gingins {'Benth.', 'Ging.'} 66
Tausch {'Tausch'} 100
Sam. {'Sam.', 'Rech.f.'} 90
Rech. {'Sam.', 'Rech.f.'}

KeyboardInterrupt: 

In [87]:
vol1_df['closest_wcvp_match'] = vol1_df.apply(get_closest_match, axis = 1)
print("vol1 complete")
vol2_df['closest_wcvp_match'] = vol2_df.apply(get_closest_match, axis = 1)
print("vol2 complete")
vol3_df['closest_wcvp_match'] = vol3_df.apply(get_closest_match, axis = 1)
print("vol3 complete")

vol1 complete
vol2 complete
vol3 complete
